<a href="https://www.kaggle.com/code/oswind/stockchat-a-stock-market-assistant?scriptVersionId=260361300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb
os.environ["ANONYMIZED_TELEMETRY"] = "false"
try:
    from kaggle_secrets import UserSecretsClient # type: ignore
except Exception as e:
    class UserSecretsClient:
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU google-genai==1.29.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2 #langgraph==0.3.21 langgraph-prebuilt==0.1.7
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core lmnr[all] browser-use
    from browser_use import Agent as BrowserAgent
else:
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-generativeai google-cloud-automl google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway
    !pip install -qU google-genai==1.29.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all]

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google import genai
from google.api_core import retry, exceptions
from google.genai import types
from IPython.display import Markdown, display
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 5.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A Gemini python api-helper with retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Gemini:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # latest: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.25,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.0-flash-001": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-preview-05-20": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-lite-06-17": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
    }
    gen_local = []
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.10 Safari/605.1.1'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.3'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: str):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = list(self.gen_model.keys()).index(default_model)
        self.default_model.append(default_model)
        self.default_local = default_model
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        self.s_embed = GeminiEmbedFunction(self.client, semantic_mode = True)
        logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0]

    def push_default_model(self, model_code: str):
        if model_code in self.gen_model.keys():
            self.stop_running()
            self.default_model.append(model_code)
            self.m_id = list(self.gen_model.keys()).index(model_code)
        else:
            print(f"{model_code} not found in gen_model.keys()")

    def pop_default_model(self):
        self.stop_running()
        self.default_model.pop(-1)
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])

    def set_default_local(self, model_index: int):
        if model_index in range(0, len(self.gen_local)):
            self.default_local = model_index
        else:
            print(f"set default local({model_index}) must be 0..{len(self.gen_local)-1}")

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model.keys())):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (genai.errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, genai.errors.APIError):
                    retriable = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if not retriable or attempt == len(self.gen_model.keys())-1:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("api.generation_fail.next_model: model is now ", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.refill_rpm ", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.zero_error: model is now ", list(self.gen_model.keys())[self.m_id])

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content)

Skipping Laminar.initialize()


In [3]:
# An embedding function based on gemini-embedding-001.
api = NewType("Gemini", None) # forward-decl
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [4]:
# Instantiate the api-helper with usage limit.
api = Gemini(with_limit=Gemini.Limit.TIER_1, default_model="gemini-2.0-flash") # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a place where shares of publicly traded companies are bought and sold.
*   It allows companies to raise capital for expansion by issuing shares to investors.
*   Investors who buy shares become part owners of the business.
*   The stock market consists of a primary market where companies issue shares and a secondary market where investors trade those shares.

**How it works:**

*   Companies issue shares through an initial public offering (IPO) to raise capital.
*   Investors buy shares hoping to receive dividends, vote in corporate elections, or sell the shares at a higher price.
*   The price of stocks rises and falls based on supply and demand.
*   Brokers facilitate transactions between buyers and sellers.

**Key Concepts:**

*   **Shares:** Represent ownership in a company.
*   **Dividends:** Payments made to shareholders from company profits.
*   **Bull Market:** A period of sustained uptrends in the stock market.
*   **Bear Market:** A period of intense downtrends in the stock market.

**Major Stock Exchanges:**

*   New York Stock Exchange (NYSE)
*   Nasdaq
*   London Stock Exchange

Keep in mind that investing in the stock market involves risk, and stock prices can be volatile.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [6]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's some information regarding AMZN (Amazon) stock that you might find helpful:

**Current Stock Status:**

*   As of September 5, 2025, Amazon (NASDAQ: AMZN) is trading around $232.84.
*   Amazon's market capitalization is approximately $2.48 trillion.
*   The stock has recovered strongly from its yearly low of $161.43, rising more than 40% in the past year.
*   The stock hit an all-time high on February 4, 2025, but experienced a downturn in March 2025 when the Nasdaq entered bear market territory.

**Analyst Ratings and Price Targets:**

*   The consensus rating among analysts is "Strong Buy".
*   Based on ratings from 45 analysts, the average price target for AMZN is $261.76, suggesting a potential increase of 12.45% over the next year.
*   The average price target from 50 analysts is $262.87.
*   Individual analyst price targets range from a low of $195 to a high of $305.
*   Analysts are generally optimistic about Amazon's prospects, particularly in the short term.

**Factors Influencing the Stock:**

*   **AWS and AI Partnerships:** Amazon Web Services (AWS) remains a significant profit driver, with revenues exceeding $120 billion annually. The partnership with Anthropic, valued at $183 billion, positions AWS as a key player in the AI space.
*   **Revenue Growth:** Amazon posted $167.7 billion in revenue in Q2 2025, a 13% increase year-over-year.
*   **Project Kuiper:** Amazon's satellite internet venture, Project Kuiper, has the potential to generate substantial revenue. JetBlue is partnering with Amazon to use Project Kuiper for in-flight Wi-Fi.
*   **AI Initiatives:** Amazon is reportedly launching a new AI-powered workspace software called Quick Suite.
*   **Grocery Expansion:** Amazon's expansion into the grocery market could potentially challenge major players like Walmart and Kroger.

**Potential Risks and Concerns:**

*   **Capital Expenditures:** Aggressive capital expenditures in AI, cloud, and satellite internet could strain free cash flow.
*   **Competition:** Amazon faces intense competition in both the retail and technology sectors.
*   **Grocery Market Margins:** The grocery business operates on thin margins, which could impact Amazon's profitability.
*   **Regulatory Concerns:** Amazon, like other large technology firms, faces increasing regulatory scrutiny.
*   **Insider Selling:** In the past three months, Amazon insiders have sold a significant amount of company stock.

**Additional Points:**

*   Amazon does not currently pay dividends.
*   The company reinvests its earnings into growth areas such as e-commerce, logistics, cloud computing (AWS), and artificial intelligence.
*   Amazon is scheduled to release its next earnings report for the period ending September 30, 2024.

**In summary,** analysts have a generally positive outlook on Amazon stock, with a "Strong Buy" consensus. They cite factors like AWS growth, AI partnerships, and Project Kuiper as potential drivers for future growth. However, it's important to consider potential risks such as high capital expenditures, competition, and regulatory concerns.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [7]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a breakdown of AMZN's current share price, short-term trends, and bullish versus bearish predictions:

**Current Share Price:**

*   As of September 5, 2025, AMZN is trading around $232.33.
*   Throughout the last trading session, the stock experienced fluctuations of 1.75%, ranging from a day low of $231.93 to a day high of $235.99.

**Short-Term Trends:**

*   **Mixed Signals:** Technical analysis suggests both bullish and bearish signals.
*   The stock closed at $235.68 on September 4, 2025, trading above its 21-day moving average of $227.59, which typically signals short-term strength.
*   The stock is in the middle of a wide and weak rising trend in the short term, and a further rise within the trend is signaled.
*   However, other sources indicate a short-term bearish trend, with the stock positioned below its 50-day moving average.
*   **Near support:** Amazon finds support from accumulated volume at $223.30, and this level may hold a buying opportunity as an upwards reaction can be expected when the support is being tested.

**Bullish Predictions:**

*   **Analyst Consensus:** The consensus rating among analysts is "Strong Buy".
*   **Price Targets:** The average analyst price target is around $261.76 - $265.00, suggesting a potential increase over the next year.
*   **Revenue Growth:** Analysts expect revenue to rise from $710 billion in 2025 to $1.153 trillion by the end of 2030.
*   **Bull Case Scenario:** Under a bull case scenario, Amazon's share price could reach $431 by 2030. This assumes AWS continues to expand, AI models propel growth, and Amazon tops Wall Street projections.
*   **Technical Breakout:** A clear technical breakout above resistance and improving fundamental metrics suggest AMZN presents a compelling long opportunity. Traders should target $242 in the near term with extended upside potential toward $252.

**Bearish Predictions:**

*   **Bear Case Scenario:** A bear case scenario projects a share price of $77 by 2030. This is based on factors like cloud competition from Microsoft Azure and unprofitable business segments.
*   **Short-Term Decline:** Some forecasts predict a short-term decline, with the value of shares potentially dropping to around $217.08 by October 4, 2025.
*   **Overvalued:** One analysis suggests it may be a bad time to buy AMZN stock because it's trading below the forecast and could be overvalued.
*   **Consolidation:** AMZN consolidates near 234. Ceiling at 242–245 is the last test before new history.

**Factors to Consider:**

*   **Market Sentiment:** The overall market sentiment is neutral, with retail investors leaning slightly bearish, while institutions are cautiously bullish.
*   **Earnings Reports:** Monitor earnings and technical breakouts for clearer direction.
*   **Macroeconomic Conditions:** Macro conditions are supportive, though tariff risks remain.
*   **Competition:** Amazon faces intense competition in both the retail and technology sectors.
*   **Regulatory Concerns:** Amazon, like other large technology firms, faces increasing regulatory scrutiny.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol for MGM Resorts International is MGM. It is listed on the New York Stock Exchange (NYSE). As of September 5, 2025, the stock is trading around $36.68.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [9]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I apologize, I am unable to provide a candlestick chart for MGM due to the error message "ModuleNotFoundError: No module named 'yfinance'". This indicates that the yfinance library, which is essential for fetching stock data, is not accessible in the current environment. Therefore, I cannot execute the Python code to generate the requested plot.


In [10]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

I am unable to generate a candlestick chart for MGM due to the error message "ModuleNotFoundError: No module named 'yfinance'". This indicates that the yfinance library, which is essential for fetching stock data, is not accessible in the current environment. Therefore, I cannot execute the Python code to generate the requested plot.


In [11]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, here's the AMZN data I mentioned earlier:

*   During the last trading session (as of September 5, 2025), AMZN experienced fluctuations of 1.75%.
*   The day's low was $231.93.
*   The day's high was $235.99.
*   The closing price on September 4, 2025, was $235.68.

I don't have the opening price for that specific day readily available.


In [12]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN (Amazon) stock data for the past month (approximately), presented in a markdown table. Keep in mind that "past month" is based on available data up to today, September 6, 2025, and the data may not be complete for the most recent days.

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| 2025-09-05 | $235.19 | $236.00 | $231.93 | $232.33 |
| 2025-09-04 | $231.19 | $235.77 | $230.78 | $235.68 |
| 2025-09-03 | $225.21 | $227.17 | $224.36 | $225.99 |
| 2025-09-02 | $223.52 | $226.17 | $221.83 | $225.34 |
| 2025-08-29 | $231.32 | $231.81 | $228.16 | $229.00 |
| 2025-08-28 | $229.01 | $232.71 | N/A     | $231.60 |
| 2025-08-27 | $228.57 | $229.87 | N/A     | $229.12 |
| 2025-08-26 | $227.11 | N/A     | N/A     | $228.71 |
| 2025-08-25 | $227.35 | N/A     | N/A     | $227.94 |
| 2025-08-22 | $222.79 | N/A     | N/A     | $228.84 |

**Disclaimer:** This data is based on information available as of September 6, 2025, and may be subject to change. N/A indicates that I don't have the data available from the search results.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

## BaseModels

In [13]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Gemini.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    V_POS = "very positive"
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"
    V_NEG = "very negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    HOLD = "hold"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        datetime_now = datetime.now(self.tz())
        self_ts = parse(self.timestamp)
        # Re-generate events for a new day.
        if datetime_now.day > self_ts.day:
            return True
        # No updates on holidays or when generated after post_close.
        if self.is_holiday or self_ts.time() >= parse(self.post_close).time():
            return False
        # Compare current time to generated event times.
        for event in [self.pre_open,self.reg_open,self.reg_close]:
            if datetime_now.time() > parse(event).time():
                return True
        # Current time is before pre_open.
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Memory

In [14]:
# Create the contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.prompt = None
        self.summary = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        subroutine_content = [types.Content(role="user", parts=[types.Part(text=prompt)]),
                              types.Content(role="model", parts=code_response_parts)]
        # Append the model's generated code and execution result.
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": subroutine_content
        }

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = None

memory = Memory()

## Retrieval-Augmented Generation Tool

In [15]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.set_holidays("US", ["09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"])
        self.generated_events("US")

    def set_holidays(self, exchange_code: str, holidays: list):
        self.holidays[exchange_code] = [datetime.strptime(h, "%m-%d-%Y").date() for h in holidays]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip("\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        gen_datetime = None
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            gen_datetime = datetime.combine(last_close_day, event_time)
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                next_event_day += timedelta(days=1)
            # Combine date and time.
            gen_datetime = datetime.combine(next_event_day, event_time)
        # Format the result as requested.
        return gen_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""What is the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""What is the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""What is the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with the time in this format: '%H:%M:%S'.
            Omit all other chat and details. Do not use sentences."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        if api.Const.Stop() in f"{response.parts[-1].text}":
            progress.close()
            api.generation_fail()
            time.sleep(api.dt_between)
            return self.generate_event(exchange_code, event)
        else:
            response = self.get_event_date(response.parts[-1].text, exchange_code, event)
            progress.update(1)
            return response

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        # Check for an existing GeneratedEvent object having updates.
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            event_obj = self.events[exchange_code]
            event_state = [(event_obj.pre_open, MarketEvent.PRE_OPEN),
                           (event_obj.reg_open, MarketEvent.REG_OPEN),
                           (event_obj.reg_close, MarketEvent.REG_CLOSE),
                           (event_obj.post_close, MarketEvent.POST_CLOSE)]
            # Need now in same format as generated.
            datetime_now = parse(datetime.now(event_obj.tz()).strftime('%c'))
            gen_ts = parse(event_obj.timestamp)
            # Re-generate events when day changes.
            if datetime_now.day > gen_ts.day:
                del self.events[exchange_code]
                return self.generated_events(exchange_code)
            # Update changed events on trading days.
            for e in event_state:
                if datetime_now > parse(e[0]):
                    event_dt = self.generate_event(exchange_code, e[1])
                    match e[1]:
                        case MarketEvent.PRE_OPEN:
                            event_obj.pre_open = event_dt
                        case MarketEvent.REG_OPEN:
                            event_obj.reg_open = event_dt
                        case MarketEvent.REG_CLOSE:
                            event_obj.reg_close = event_dt
                        case MarketEvent.POST_CLOSE:
                            event_obj.post_close = event_dt
            event_obj.timestamp = datetime.now(event_obj.tz()).strftime('%c')
            self.events[exchange_code] = event_obj
        # Generate events for an exchange code not in cache.
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE),
                is_holiday=datetime.now().date() in self.holidays[exchange_code])
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Gemini.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wikipedia Search Tool

In [16]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Gemini.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        page_html = api.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(page_html)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return StopGeneration().result

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Google Search Tool

In [17]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return StopGeneration().result # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches[0]
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest API Tool and Helpers

In [18]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",50
    POLY = "polygon.io",4 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [19]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(api.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = api.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return StopGeneration().result

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return StopGeneration().result

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return StopGeneration().result

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

# Instantiate the Tools

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate US->MarketEvent.LAST_CLOSE:   0%|          | 0/1 [00:00<?, ?it/s]


api.generation_fail.next_model: model is now  gemini-2.5-flash


Generate US->MarketEvent.LAST_CLOSE:   0%|          | 0/1 [00:02<?, ?it/s]


api.generation_fail.next_model: model is now  gemini-2.5-pro


Generate US->MarketEvent.LAST_CLOSE:   0%|          | 0/1 [00:10<?, ?it/s]


api.generation_fail.next_model: model is now  gemini-2.5-flash-lite


Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:00<00:00,  1.73it/s]
Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [21]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.candidates[0].content.parts[-1].text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.candidates[0].content.parts[-1].text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```json
{
  "VN": "Vietnam",
  "AD": "ABU DHABI SECURITIES EXCHANGE",
  "US": "US exchanges (NYSE, Nasdaq)",
  "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
  "QA": "QATAR EXCHANGE",
  "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
  "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
  "PR": "PRAGUE STOCK EXCHANGE",
  "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
  "CA": "Egyptian Stock Exchange",
  "AX": "ASX - ALL MARKETS",
  "SX": "DEUTSCHE BOERSE Stoxx",
  "KQ": "KOREA EXCHANGE (KOSDAQ)",
  "DB": "DUBAI FINANCIAL MARKET",
  "PM": "Philippine Stock Exchange",
  "KS": "KOREA EXCHANGE (STOCK MARKET)",
  "ST": "NASDAQ OMX NORDIC STOCKHOLM",
  "DU": "BOERSE DUESSELDORF",
  "TL": "NASDAQ OMX TALLINN",
  "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
  "SW": "SWISS EXCHANGE",
  "LS": "NYSE EURONEXT - EURONEXT LISBON",
  "SI": "SINGAPORE EXCHANGE",
  "RG": "NASDAQ OMX RIGA",
  "CR": "CARACAS STOCK EXCHANGE",
  "SA": "Brazil Bolsa - Sao Paolo",
  "BH": "BAHRAIN BOURSE",
  "NZ": "NEW ZEALAND E

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [22]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It provides the last trading days' data after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It includes 
                   the last trading days' data after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calling Expert

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

## Callable Functions and Handler

In [23]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != StopGeneration().result:
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return StopGeneration().result
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

## Define the System Prompt

In [24]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

## Import the Rest API Keys

In [25]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

## The Function Caller

In [26]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    response = api.retriable(api.client.models.generate_content,
                             model=api(Gemini.Model.GEN),
                             config=config_fncall,
                             contents=memory.contents)
    while function_calling_in_process:
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                try:
                    api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                except KeyError as e: # Gemini sometimes omits required fn_args
                    api.generation_fail()
                    time.sleep(api.dt_between)
                    send_message(prompt)
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                memory.update_contents(function_call, api_response_part)
                # Send the updated prompt.
                response = api.retriable(api.client.models.generate_content,
                                         model=api(Gemini.Model.GEN),
                                         config=config_fncall,
                                         contents=memory.contents)
            else:
                # Response may be a summary or reasoning step.
                if len(response.candidates[0].content.parts) == 1:
                    function_calling_in_process = False
                    memory.set_summary(response.text.replace("$", "\\$"))
                    break # No more parts in response.
                else:
                    #display(Markdown("#### Natural language reasoning step"))
                    #print(response)
                    memory.set_reason(response.candidates[0].content.parts[0].text)
                    continue # Next part contains a function call.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

api.zero_error: model is now  gemini-2.0-flash


# Ask a question

<span style="font-size:18px;">
    If you're on free-tier of Gemini you probably want to Run-before here. Your usage tier can be configured in the api-helper at the top of the notebook.
</span>

In [27]:
send_message("What is the current session for US exchanges?")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

The current session for US exchanges is closed.


In [29]:
send_message("What is the US market status?")

#### Natural language response

The US market is currently closed. The last status update was on Sat, 06 Sep 2025 12:12:10 EDT.


In [30]:
send_message("When was the last US market close?")

#### Natural language response

The last US market close was Fri Sep 05 20:00:00 2025.


In [31]:
send_message("What is Apple's stock ticker?")

Score similarity to query: 100%|██████████| 10/10 [00:00<00:00, 11.19it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Apple's stock ticker is AAPL.


In [32]:
send_message("What is the current price of Amazon stock? Display the result as a json object.")

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is the current price of Amazon stock:
```json
{
"c": 232.33,
"d": -3.35,
"dp": -1.4214,
"h": 236.0,
"l": 231.93,
"o": 235.19,
"pc": 235.68,
"t": 1757102400
}
```
The current price is \$232.33 as of Fri Sep  5 16:00:00 2025. The price changed by -3.35, which is -1.4214 percent. The high price of the day is \$236.0, and the low price of the day is \$231.93. The open price of the day is \$235.19, and the previous close price is \$235.68.


api.refill_rpm  2000


In [33]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
                How has the stock performed?""")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's an overview of Apple's financial performance based on the data:

**Financial Health & Performance:**
*   **Revenue Growth:** Apple has demonstrated consistent revenue growth over the past 3 and 5 years, with quarterly and TTM (trailing twelve months) growth indicating continued positive momentum.
*   **Profitability:** Apple maintains strong profitability margins, including gross margin, operating margin, and net profit margin.
*   **Asset Turnover:** The asset turnover ratio indicates how efficiently Apple is using its assets to generate revenue.
*   **Return on Equity (ROE):** Apple exhibits a high ROE, suggesting efficient use of equity financing.
*   **Debt Management:** Apple's debt-to-equity ratio and debt-to-total asset ratio provide insights into the company's financial leverage.
*   **Earnings Per Share (EPS):** Apple's EPS has generally increased over the years.

**Stock Performance Indicators:**
*   **52 Week Performance:** The 52-week price return daily is 8.5307. The 52-week high is 260.1, and the 52-week low is 169.2101.
*   **Price-to-Earnings Ratio (P/E):** The P/E ratio is a valuation metric that compares Apple's stock price to its earnings per share.
*   **Price-to-Book Ratio (P/B):** The P/B ratio compares Apple's market capitalization to its book value.
*   **Price-to-Sales Ratio (P/S):** The P/S ratio compares Apple's market capitalization to its revenue.
*   **Beta:** Beta is a measure of Apple's stock price volatility relative to the overall market.

**Additional Key Metrics:**
*   **Dividend Yield:** The current dividend yield TTM is 0.456.
*   **Cash Flow:** Apple's cash flow per share and free cash flow metrics provide insights into the company's ability to generate cash.
*   **Liquidity Ratios:** Apple's current ratio and quick ratio indicate its ability to meet short-term obligations.


In [34]:
send_message("I need Apple's daily candlestick from 2025-05-05")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

OK. On 2025-05-05, Apple's stock (AAPL) had the following daily candlestick data:
- Open: 203.1
- High: 204.1
- Low: 198.21
- Close: 198.89
- Volume: 69018452
- Pre-Market: 205.0
- After Hours: 198.6


In [51]:
send_message("Tell me who are Apple's peers?")

#### Natural language response

Apple's peers include: DELL TECHNOLOGIES -C (DELL), WESTERN DIGITAL CORP (WDC), HEWLETT PACKARD ENTERPRISE (HPE), HP INC (HPQ), PURE STORAGE INC - CLASS A (PSTG), SUPER MICRO COMPUTER INC (SMCI), NETAPP INC (NTAP), IONQ INC (IONQ), SANDISK CORP (SNDK), QUANTUM COMPUTING INC (QUBT), and DIEBOLD NIXDORF INC (DBD).


In [52]:
send_message("Tell me who are Amazon's peers?")

#### Natural language response

Amazon's peers include COUPANG INC (CPNG), EBAY INC (EBAY), DILLARDS INC-CL A (DDS), OLLIE'S BARGAIN OUTLET HOLDI (OLLI), ETSY INC (ETSY), MACY'S INC (M), SAVERS VALUE VILLAGE INC (SVV), KOHLS CORP (KSS), GROUPON INC (GRPN), and CONTEXTLOGIC HOLDINGS INC (LOGC).


api.refill_rpm  2000


In [53]:
send_message(
    """Locate Apple's stock ticker, then download recommendation trends of all Apple's peers by sub-industry, 
    and then finally compare them.""")

Upsert chunks embedding: 0it [00:00, ?it/s]


api.refill_rpm  150


Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Of Apple's peers, DELL, WDC, and PSTG have the best analyst ratings, while SMCI and HPQ have the worst.

| Ticker | Strong Buy | Buy | Hold | Sell | Strong Sell |
|---|---|---|---|---|---|
| DELL | 7 | 17 | 6 | 0 | 0 |
| WDC | 6 | 19 | 6 | 0 | 0 |
| HPE | 6 | 8 | 10 | 0 | 0 |
| HPQ | 2 | 4 | 14 | 1 | 0 |
| PSTG | 6 | 12 | 8 | 1 | 0 |
| SMCI | 2 | 10 | 10 | 3 | 0 |
| NTAP | 3 | 9 | 15 | 0 | 0 |
| IONQ | 2 | 9 | 3 | 0 | 0 |
| SNDK | 6 | 10 | 8 | 0 | 0 |
| QUBT | 2 | 4 | 2 | 0 | 0 |
| DBD | 2 | 4 | 1 | 0 | 0 |



api.refill_rpm  150


In [54]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.
    Discuss and provide details about any patterns you notice in the price data.
    Correlate recent patterns with news over the same date range.""")

Upsert chunks embedding: 0it [00:00, ?it/s]


api.refill_rpm  150


#### Natural language response

Amazon's current share price is \$232.33.

### Candlestick Data for the Past Month (Sorted in Descending Order)

| Date | Open | High | Low | Close |
|---|---|---|---|---|
| 2025-09-05 | \$235.19 | \$236.00 | \$231.93 | \$232.33 |
| 2025-09-04 | \$231.19 | \$235.77 | \$230.78 | \$235.68 |
| 2025-09-03 | \$225.21 | \$227.17 | \$224.36 | \$225.99 |
| 2025-09-02 | \$223.52 | \$226.17 | \$221.83 | \$225.34 |
| 2025-08-29 | \$231.32 | \$231.81 | \$228.16 | \$229.00 |
| 2025-08-28 | \$229.01 | \$232.71 | \$228.02 | \$231.60 |
| 2025-08-27 | \$228.57 | \$229.87 | \$227.81 | \$229.12 |
| 2025-08-26 | \$227.11 | \$229.00 | \$226.02 | \$228.71 |
| 2025-08-25 | \$227.35 | \$229.60 | \$227.31 | \$227.94 |
| 2025-08-22 | \$222.79 | \$229.14 | \$220.82 | \$228.84 |
| 2025-08-21 | \$222.65 | \$222.78 | \$220.50 | \$221.95 |
| 2025-08-20 | \$227.12 | \$227.27 | \$220.92 | \$223.81 |
| 2025-08-19 | \$230.09 | \$230.53 | \$227.12 | \$228.01 |
| 2025-08-18 | \$230.23 | \$231.91 | \$228.33 | \$231.49 |
| 2025-08-15 | \$232.58 | \$234.08 | \$229.81 | \$231.03 |
| 2025-08-14 | \$227.40 | \$233.11 | \$227.02 | \$230.98 |
| 2025-08-13 | \$222.00 | \$224.92 | \$222.00 | \$224.56 |
| 2025-08-12 | \$222.23 | \$223.50 | \$219.05 | \$221.47 |
| 2025-08-11 | \$221.78 | \$223.05 | \$220.40 | \$221.30 |
| 2025-08-08 | \$223.14 | \$223.80 | \$221.88 | \$222.69 |
| 2025-08-07 | \$221.00 | \$226.22 | \$220.82 | \$223.13 |
| 2025-08-06 | \$214.70 | \$222.65 | \$213.74 | \$222.31 |

### Price Data Patterns and News Correlation

The candlestick data for the past month shows a general upward trend in Amazon's stock price, with some periods of volatility. The stock started the month at \$214.70 and ended at \$232.33, representing a significant gain.

This upward trend is strongly correlated with a wealth of positive news surrounding the company. The news sentiment for Amazon over the past month has been overwhelmingly positive, with numerous reports highlighting the company's strengths and growth prospects.

Several key themes emerge from the news that likely contributed to the stock's positive performance:

*   **Artificial Intelligence (AI) Advancements:** Many news articles focused on Amazon's innovations in AI, including the development of a new "Quick Suite" AI-powered workspace software and the company's continued investment in AI infrastructure. This has generated significant investor confidence in Amazon's ability to compete in the rapidly growing AI sector.
*   **E-commerce Expansion:** News of Amazon expanding its same-day grocery delivery service and forming new partnerships, such as the one with Hertz to sell used cars, has reinforced the company's dominance in the e-commerce space.
*   **Strong Financial Performance and Analyst Ratings:** Reports of strong earnings, bullish analyst ratings, and price targets suggesting significant upside potential have further fueled investor optimism.
*   **Cloud Computing Growth:** Despite some concerns about a slowdown in AWS growth, the overall sentiment remains positive, with Amazon's cloud division continuing to be a major profit driver.

In conclusion, the positive news flow, particularly around Amazon's AI initiatives and e-commerce expansion, has likely been a major catalyst for the stock's upward momentum over the past month. The consistent positive sentiment from analysts and the company's strong financial performance have also played a crucial role in boosting investor confidence.

In [55]:
send_message("What is Apple's ticker overview")

#### Natural language response

Apple Inc. (AAPL) is a publicly traded company listed on the XNAS exchange. It operates in the stocks market and its currency is the USD. The company has a market capitalization of \$3,557,093,079,100.00.

Here are some key details about Apple Inc.:

*   **CIK:** 0000320193
*   **Composite FIGI:** BBG000B9XRY4
*   **Share Class FIGI:** BBG001S5N8V8
*   **Phone Number:** (408) 996-1010
*   **Address:** ONE APPLE PARK WAY, CUPERTINO, CA 95014
*   **Description:** Apple is among the largest companies in the world, with a broad portfolio of hardware and software products targeted at consumers and businesses. Apple's iPhone makes up a majority of the firm sales, and Apple's other products like Mac, iPad, and Watch are designed around the iPhone as the focal point of an expansive software ecosystem. Apple has progressively worked to add new applications, like streaming video, subscription bundles, and augmented reality. The firm designs its own software and semiconductors while working with subcontractors like Foxconn and TSMC to build its products and chips. Slightly less than half of Apple's sales come directly through its flagship stores, with a majority of sales coming indirectly through partnerships and distribution.
*   **SIC Code:** 3571 (ELECTRONIC COMPUTERS)
*   **Homepage:** https://www.apple.com
*   **Total Employees:** 164,000
*   **List Date:** 1980-12-12
*   **Branding:**
    *   Logo URL: https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_logo.svg
    *   Icon URL: https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_icon.png
*   **Share Information:**
    *   Share Class Shares Outstanding: 14,840,390,000
    *   Weighted Shares Outstanding: 14,840,390,000
    *   Round Lot: 100

In [56]:
send_message("Tell me about Amazon's historical and current recommendation trends")

#### Natural language response

Here are the recommendation trends for Amazon (AMZN):

September 2025:
- Strong Buy: 23
- Buy: 52
- Hold: 4
- Sell: 0
- Strong Sell: 0

August 2025:
- Strong Buy: 24
- Buy: 51
- Hold: 4
- Sell: 0
- Strong Sell: 0

July 2025:
- Strong Buy: 24
- Buy: 50
- Hold: 5
- Sell: 0
- Strong Sell: 0

June 2025:
- Strong Buy: 24
- Buy: 50
- Hold: 5
- Sell: 0
- Strong Sell: 0

In [57]:
send_message("What is Google's stock ticker symbol?")

api.refill_rpm  150


#### Natural language response

The stock ticker symbols for Google are GOOGL and GOOG, and they are listed on the NASDAQ stock exchange. These symbols now refer to Alphabet Inc., which is Google's holding company. The company is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1.

In [58]:
send_message("What is MGM Studio's stock symbol?")

api.refill_rpm  150


#### Natural language response

I am sorry, but I cannot find a stock symbol for MGM Studios. It is possible that it is not a publicly traded company.

In [59]:
send_message("What is MGM Studio's owner company stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


api.refill_rpm  150


Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

MGM Studio is owned by Amazon, and its stock symbol is AMZN.

In [60]:
send_message("What is Facebook's stock ticker symbol?")

#### Natural language response

The stock ticker symbol for Facebook is META.

In [61]:
send_message('''Compare Amazon's bullish versus bearish predictions from July 01 2025 until today.
                Include a discussion of recommendation trends, and sentiment analysis of news from the same dates.
                Discuss any patterns or correlations you find.''')

api.refill_rpm  150


#### Natural language response

Here is a comparison of Amazon's bullish versus bearish predictions from July 01, 2025, to September 06, 2025, including a discussion of recommendation trends and sentiment analysis of news from the same period.

### Recommendation Trends

Analyst recommendations for Amazon (AMZN) from July to September 2025 show a consistently strong bullish sentiment. The number of "strong buy" and "buy" recommendations remained overwhelmingly positive, with very few "hold" recommendations and no "sell" or "strong sell" ratings.

| Period | Strong Buy | Buy | Hold | Sell | Strong Sell |
|---|---|---|---|---|---|
| **2025-07-01** | 24 | 50 | 5 | 0 | 0 |
| **2025-08-01** | 24 | 51 | 4 | 0 | 0 |
| **2025-09-01** | 23 | 52 | 4 | 0 | 0 |

This data indicates a stable and overwhelmingly positive outlook from analysts, with a slight increase in "buy" recommendations and a slight decrease in "strong buy" and "hold" recommendations over the three-month period. This suggests that while the overall sentiment remains bullish, there might be a minor shift in the intensity of that sentiment.

### Sentiment Analysis of News

The sentiment from news articles about Amazon during this period was also predominantly positive, reinforcing the bullish analyst recommendations. Here is a summary of the key themes and sentiments found in the news:

**Bullish Sentiment:**

*   **Strong Financial Performance:** Many articles highlighted Amazon's strong financial performance, including revenue growth, profitability, and the success of its various business segments like AWS, advertising, and e-commerce.
*   **AI and Cloud Dominance:** A significant number of articles focused on Amazon's leadership in cloud computing with AWS and its heavy investments in Artificial Intelligence. The potential for AI to drive future growth was a recurring theme.
*   **Market Leadership and Expansion:** News frequently mentioned Amazon's dominant market position and its continuous expansion into new areas like grocery delivery, robotics, and healthcare.
*   **Positive Analyst Outlook:** Several articles reported on the bullish price targets and "buy" ratings from Wall Street analysts, further reinforcing the positive sentiment.

**Bearish/Neutral Sentiment:**

*   **Slower AWS Growth:** Some articles pointed to a slowdown in the growth rate of AWS, especially when compared to competitors like Microsoft Azure. This was a recurring theme in more neutral or slightly bearish articles.
*   **Valuation Concerns:** A few articles raised concerns about Amazon's high valuation and whether the stock price was getting ahead of its earnings.
*   **Tariffs and Economic Headwinds:** The potential impact of tariffs and broader economic challenges on Amazon's business was mentioned in some articles, though this was not a dominant theme.
*   **Competition:** While Amazon's market leadership was acknowledged, the growing competition from other tech giants in areas like cloud and AI was also a point of discussion in some articles.

### Patterns and Correlations

There is a strong correlation between the bullish analyst recommendations and the positive sentiment found in the news. Both data sources point to a prevailing optimism about Amazon's future prospects.

*   The consistent "strong buy" and "buy" ratings from analysts are mirrored by the frequent news reports on Amazon's strong financial performance, market leadership, and AI-driven growth potential.
*   The minor concerns raised in some news articles, such as the slowing growth of AWS, are reflected in the small number of "hold" recommendations from analysts. This suggests that while the overall outlook is positive, there are some factors that are giving a few analysts pause.
*   There is no significant bearish sentiment in either the analyst recommendations or the news analysis, indicating a broad consensus that Amazon's strengths far outweigh its weaknesses.

In conclusion, the period from July to September 2025 was characterized by a strong and stable bullish outlook for Amazon. Both analyst recommendations and news sentiment were overwhelmingly positive, with only minor concerns being raised. The primary drivers of this optimism were Amazon's strong financial performance, its leadership in cloud and AI, and its continued expansion into new markets.

api.refill_rpm  150


In [62]:
send_message('''Compare Google's bullish versus bearish predictions from July 01 2025 until today.
                Include a discussion of recommendation trends, and sentiment analysis of news from the same dates.
                Discuss any patterns or correlations you find.''')

Upsert chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


api.refill_rpm  150


Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is a comparison of Google's bullish versus bearish predictions from July 1, 2025, to September 6, 2025, including a discussion of recommendation trends, sentiment analysis of news, and any observed patterns or correlations.

### Bullish vs. Bearish Predictions

Overall, the sentiment surrounding Google (GOOGL) from July to early September 2025 has been predominantly bullish. This is reflected in analyst recommendations, news sentiment, and the stock's price performance.

### Recommendation Trends

Analyst recommendations show a clear bullish trend for Google during this period. The number of "buy" recommendations increased, while "strong buy" and "hold" recommendations remained stable. There were no "sell" or "strong sell" recommendations, indicating a strong consensus among analysts that Google is a solid investment.

| Period | Strong Buy | Buy | Hold | Sell | Strong Sell |
|---|---|---|---|---|---|
| 2025-09-01 | 18 | 37 | 13 | 0 | 0 |
| 2025-08-01 | 18 | 37 | 13 | 0 | 0 |
| 2025-07-01 | 18 | 32 | 13 | 0 | 0 |

### News Sentiment Analysis

News sentiment analysis from July 1 to September 6, 2025, reveals a largely positive outlook for Google, with some notable bearish points.

**Bullish News:**

*   **Strong Financial Performance:** Many news articles highlighted Google's strong financial performance, particularly in its cloud and search segments.
*   **AI Advancements:** The successful integration of AI into its products, such as Google Search and Google Cloud, has been a significant driver of positive sentiment.
*   **Favorable Antitrust Ruling:** A key positive event was the favorable antitrust ruling in early September, which removed a significant overhang for the stock and led to a surge in its price.
*   **Strategic Partnerships:** Positive news also came from strategic partnerships, such as the collaboration with Apple to use Google's Gemini AI for Siri.

**Bearish News:**

*   **Competition:** Some articles expressed concerns about increasing competition in the AI space from other tech giants.
*   **Regulatory Risks:** The ongoing antitrust scrutiny, despite the favorable ruling, remains a potential headwind.
*   **Data Privacy:** A few articles raised concerns about data privacy issues, which could lead to regulatory action and reputational damage.

### Patterns and Correlations

Several patterns and correlations can be observed from the data:

*   **Positive Correlation between News Sentiment and Stock Price:** There is a strong positive correlation between news sentiment and Google's stock price. Positive news, such as the favorable antitrust ruling, was immediately followed by a significant increase in the stock price.
*   **Analyst Recommendations Reflecting Positive Momentum:** The increase in "buy" recommendations from analysts in August and September aligns with the positive news flow and the stock's upward trend.
*   **Overall Bullish Trend:** The combination of positive analyst ratings, predominantly positive news sentiment, and a rising stock price indicates a strong bullish trend for Google during this period. The bearish sentiment, while present, was not significant enough to derail the overall positive momentum.

In conclusion, the period from July to early September 2025 was very positive for Google. The company's strong financial performance, advancements in AI, and a favorable legal development contributed to a bullish outlook from both analysts and the media, which was reflected in the stock's strong performance.

api.refill_rpm  150


In [64]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment from July 01 2025 until today?
    Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.''')

api.refill_rpm  150


#### Natural language response

Based on my analysis, here's the outlook for Apple and its peers from July 01, 2025, to September 6, 2025.

### Apple (AAPL) Outlook

**Recommendation Trends:**
Apple's analyst recommendations have remained largely positive. In the last four months, the number of "buy" and "strong buy" recommendations has consistently outnumbered "hold," "sell," and "strong sell" ratings.
- **July 2025:** 24 buy, 14 strong buy, 14 hold, 3 sell, 1 strong sell.
- **August 2025:** 24 buy, 14 strong buy, 15 hold, 3 sell, 0 strong sell.
- **September 2025:** 23 buy, 14 strong buy, 15 hold, 3 sell, 0 strong sell.

**News Sentiment:**
The news sentiment for Apple during this period has been mixed, with a significant number of articles having a negative or neutral sentiment.

- **Positive:** Several articles highlighted Apple's strong financial performance, including robust App Store revenue, a \$100 billion U.S. manufacturing investment to avoid tariffs, and a new multiyear supply partnership with Coherent for next-generation VCSEL products. The company's strategic partnerships, such as the one with MP Materials for recycled rare earth magnets, have also been viewed positively.
- **Negative:** A recurring theme in the negative news has been the delay in the development of Apple Intelligence and Siri AI features, leading to a stock price drop and multiple securities fraud lawsuits. Concerns about a lag in AI advancements, stagnant growth, and high valuation have also been raised by analysts and investors, including Warren Buffett, who has been reducing his stake in the company.
- **Neutral:** Many articles took a neutral stance, acknowledging both Apple's challenges and its strengths. These reports often pointed to the company's strong brand, loyal customer base, and robust ecosystem as factors that could help it overcome its current difficulties.

### Peer Analysis

Here's a summary of the outlook for Apple's peers in the Computers and Peripherals sub-industry:

*   **Dell Technologies (DELL):** News sentiment for Dell has been mixed. Positive news includes the company being recognized for its channel performance and the launch of new AI-focused PCs. However, there are also concerns about a potential slowdown in PC sales and the company's high valuation.
*   **Western Digital (WDC):** The sentiment for Western Digital has been largely neutral to positive. The company has been focusing on its new consumer brand identity and has seen strong performance in its flash business.
*   **Hewlett Packard Enterprise (HPE):** HPE has a generally positive outlook, with a focus on its AI solutions and GreenLake platform. The company has been expanding its partnerships and has seen strong demand for its AI-native offerings.
*   **HP Inc. (HPQ):** HP has a mixed to positive outlook. The company has been recognized for its channel performance and has been focusing on the AI PC market. However, there are some concerns about the broader PC market.
*   **Pure Storage (PSTG):** Pure Storage has a very positive outlook, with strong earnings, revenue growth, and a focus on its AI-powered platform. The company has been consistently outperforming expectations.
*   **Super Micro Computer (SMCI):** Super Micro has a positive outlook, driven by the demand for its AI servers. The company has been expanding its manufacturing capacity and has a strong relationship with Nvidia.
*   **NetApp (NTAP):** NetApp has a positive outlook, with a focus on its all-flash storage solutions and cloud services. The company has been seeing strong growth and has been upgrading its full-year guidance.
*   **IonQ (IONQ):** IonQ has a very positive outlook, with significant advancements in its quantum computing technology. The company has been achieving its technical milestones ahead of schedule and has a strong pipeline of potential commercial applications.
*   **Quantum Computing Inc. (QUBT):** Quantum Computing Inc. has a positive outlook, with a focus on its quantum optimization platform. The company has been securing new contracts and expanding its government and commercial business.
*   **Diebold Nixdorf (DBD):** Diebold Nixdorf has a mixed outlook. The company has been undergoing a financial restructuring and has been focusing on its new DN Series ATMs. While there are signs of a turnaround, the company still faces challenges.

### Comparison and Conclusion

Overall, the outlook for Apple is mixed. While the company's financial performance remains strong and analyst recommendations are generally positive, there are significant concerns about its AI strategy and innovation pipeline. The multiple lawsuits and the fact that a major investor like Warren Buffett is reducing his stake are also causes for concern.

In comparison, many of Apple's peers, particularly those in the AI and quantum computing spaces like **IonQ**, **Pure Storage**, and **Super Micro Computer**, have a more positive outlook. These companies are at the forefront of technological innovation and are seeing strong demand for their products and services.

While Apple's strong brand and ecosystem should not be underestimated, the company will need to address the concerns about its AI strategy to maintain its leadership position in the tech industry. Investors will be closely watching the upcoming iPhone 17 event and any announcements related to Apple Intelligence for signs of a clear path forward.

In [65]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? From 2025-07-01 up to today.
    Also locate candlestick data for the same dates. 
    Discuss in detail any correlations in patterns between the candlestick and news data.
    Ignore duplicate news entry.''')

#### Natural language response

Recent news about Apple (AAPL) from July 1, 2025, to September 6, 2025, has been heavily focused on the impact of tariffs, the company's AI strategy, and legal challenges. An analysis of this news in conjunction with the stock's candlestick data reveals several interesting correlations.

### Candlestick Data Analysis (July 1, 2025 - September 6, 2025)

The candlestick data for Apple (AAPL) during this period shows a significant amount of volatility, with a general upward trend in the latter half of the period.

*   **Early to Mid-July:** The stock experienced a period of decline and stagnation, with several days of significant downward movement. For example, on July 14th, the stock saw a notable drop.
*   **Late July to Early August:** The stock began to show signs of recovery, with some upward momentum.
*   **Mid-August:** A strong upward trend is visible, with a significant price jump around August 7th and 8th. This was followed by a period of sustained gains.
*   **Late August to Early September:** The stock continued to trade at a higher price point, with some fluctuations.

### News and Tariff Impact Analysis

The news surrounding Apple during this period was a mix of positive, negative, and neutral sentiment, with a significant focus on tariffs and the company's efforts to mitigate their impact.

**Key News Events and Their Correlation with Stock Performance:**

*   **Early July - Negative Sentiment and Stock Decline:**
    *   **July 1, 2025:** An article titled "Apple Stock Had A Nightmare Year — But It May Be Nearing A Turning Point" highlighted the stock's struggles.
    *   **July 6, 2025:** Another article discussed how poor earnings quality and high stock market valuations pose a significant risk to investors.
    *   **Correlation:** This period of negative news corresponds with the stock's decline and stagnation in early to mid-July. The candlestick for July 14th, in particular, shows a significant drop, which aligns with the negative sentiment from the news around that time.

*   **Mid-July - Mixed News and Volatility:**
    *   **July 15, 2025:** Positive news about Apple's \$500 million investment in MP Materials to secure a domestic supply of rare earth magnets.
    *   **July 13, 14, 17, 26, 30, 2025:** A series of negative news articles about a securities fraud lawsuit related to Apple's AI development.
    *   **Correlation:** This mix of positive and negative news is reflected in the stock's volatility during this period. The positive news about the MP Materials partnership may have contributed to some upward price movement, but the persistent negative news about the lawsuit likely tempered any significant gains.

*   **Early to Mid-August - Positive News and Strong Rally:**
    *   **August 7, 2025:** A very positive article, "Apple: Why the Stock Is Protected From Trump Admin’s Semiconductor Tariff Plans," announced a \$100 billion U.S. manufacturing commitment to secure an exemption from potential semiconductor tariffs.
    *   **August 12, 2025:** Another positive article, "Apple Stock Jumps on Trump Policy Wins — But AI Doubts Remain," highlighted the stock's rise after the investment announcement.
    *   **August 19, 2025:** Positive news about Apple achieving a milestone by producing all iPhone 17 models in India, further mitigating tariff impacts.
    *   **Correlation:** This string of positive news, particularly the announcement of the tariff exemption, directly correlates with the strong upward trend in the stock price in mid-August. The candlestick data shows a significant price jump around August 7th and 8th, which aligns perfectly with the timing of this news.

*   **Late August to Early September - Continued Strength with Some Concerns:**
    *   **August 29, 2025:** News that Warren Buffett's Berkshire Hathaway was continuing to trim its Apple stake.
    *   **September 2, 2025:** A positive article suggesting Apple is on the verge of an AI-driven supercycle.
    *   **September 3, 2025:** News of a lawsuit from Elon Musk's xAI against Apple and OpenAI.
    *   **September 6, 2025:** An article suggesting that Apple's stock is overvalued.
    *   **Correlation:** The stock maintained its higher price point during this period, suggesting that the positive sentiment from the tariff exemption and production diversification news continued to outweigh concerns about the lawsuit and valuation. However, the news about Berkshire Hathaway selling shares and the overvaluation concerns may have contributed to some of the price fluctuations seen in late August and early September.

### Detailed Correlation Patterns

1.  **Tariff News as a Primary Driver:** The most significant correlation observed is between the news related to tariffs and the stock's performance. Negative news about potential tariffs in early July corresponded with a stock price decline, while the positive news of a tariff exemption in early August triggered a strong and sustained rally. This suggests that investors were highly sensitive to news about the impact of tariffs on Apple's business.

2.  **AI Development as a Secondary Factor:** News about Apple's AI strategy, both positive (potential acquisitions and a future "supercycle") and negative (lawsuits and perceived lag in development), also appears to have influenced the stock price, but to a lesser extent than the tariff news. The negative news about the AI lawsuit in mid-July likely contributed to the stock's volatility, but it was not enough to cause a major downturn.

3.  **Investor Sentiment and Market Narratives:** The news articles also reflect broader market narratives about Apple's valuation, growth prospects, and the impact of Warren Buffett's investment decisions. These narratives can create a general sentiment that influences investor behavior, even in the absence of specific, market-moving news. For example, the repeated articles about Berkshire Hathaway selling its stake may have created some underlying selling pressure on the stock.

### Conclusion

In conclusion, the recent news about Apple and the impact of tariffs had a clear and direct correlation with the stock's candlestick data from July 1, 2025, to September 6, 2025. The company's strategic moves to mitigate tariff risks, such as the \$100 billion U.S. manufacturing investment and the diversification of production to India, were met with a strong positive reaction from the market, leading to a significant stock price increase. While other factors, such as the company's AI strategy and legal challenges, also played a role, the news related to tariffs was the primary driver of the stock's performance during this period. This analysis highlights the importance of monitoring geopolitical and trade-related news when investing in multinational companies like Apple.

api.refill_rpm  150


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>

# Update June 7, 2025

<span style="font-size:18px;">
    Bugfix version 102 finally brings Essy to a stable milestone. A month and a half late :) There's still more to be built including adding reasoning, agents, and structured output. A few unimplemented rest endpoints remain that could make Essy more self-reliant. The vector store has gotten bigger but not smarter. Essy can tell us pre-scored news has some sentiment but cannot generate it due to limited summaries. Essy can detect interesting patterns in a dataset but not between adjacent datasets. There's so much data we'll need to recruit Essy some help.
</span>

# Advanced (localhost required)

<span style="font-size:18px;">
    The functions demonstrated here require a locally running notebook. A dedicated GPU with at least 8GB VRAM is recommended but not required. Output is generated with Gemma 3 12B QAT, Gemma.cpp, and (later) Gemma 3n. Output on Kaggle is based on cached data.
</span>

In [ ]:
# soon